# Mosaic Augmentation을 적용 후 동작 확인 과정 코드

In [ ]:
import cv2 
import os

import matplotlib.pyplot as plt
import matplotlib.patches as patches

from tqdm import tqdm

from pycocotools.coco import COCO

In [1]:
coco = COCO('/opt/ml/input/data/train_mosaic_upsampling.json')

In [2]:
len(coco.getImgIds())

In [42]:
image_id = coco.getImgIds()

In [43]:
# image_id

In [44]:
for i in range(0, 3446):
    coco.loadImgs(i)[0]

image_infos = coco.loadImgs(3446)[0]

In [3]:
image_infos

In [46]:
ann_ids = coco.getAnnIds(imgIds=3446)

In [47]:
anns = coco.loadAnns(ann_ids)

In [4]:
len(anns)

In [49]:
cat_ids = coco.getCatIds()

In [5]:
cat_ids

In [51]:
cats = coco.loadCats(cat_ids)

In [6]:
len(cats)

In [7]:
cats[0]

In [54]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

In [55]:
nms = [cat['name'] for cat in cats]

In [8]:
print('COCO categories: \n{}\n'.format(' '.join(nms)))

In [9]:
nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

In [58]:
catIds = coco.getCatIds(catNms = ['Battery'])

In [10]:
catIds

In [60]:
imgIds = coco.getImgIds(catIds= catIds)

In [11]:
imgIds

In [63]:
image_infos = coco.loadImgs(3446)[0]

In [12]:
image_infos

In [13]:
root_path = "/opt/ml/input/data/"

I = io.imread(root_path + image_infos['file_name'])
plt.axis('off')
plt.imshow(I)
plt.show()

In [14]:
catIds = coco.getCatIds(catNms=['Battery']);
imgIds = coco.getImgIds(catIds=catIds);
print(catIds)
print(imgIds)

In [67]:
def get_img_label(index, img_size, coco, data_dir):
        image_id = coco.getImgIds(imgIds=index)
        
        image_info = coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        ann_ids = coco.getAnnIds(imgIds=image_info['id'])
        anns = coco.loadAnns(ann_ids)
        
        boxes = np.array([x['bbox'] for x in anns])
#         boxes = [x['bbox'] for x in anns]

        # boxex (x_min, y_min, x_max, y_max)
        boxes[:, 2] = (boxes[:, 0] + boxes[:, 2]) * img_size/ image.shape[1]
        boxes[:, 3] = (boxes[:, 1] + boxes[:, 3]) * img_size / image.shape[0]
        boxes[:, 0] = boxes[:, 0] * img_size / image.shape[1]
        boxes[:, 1] = boxes[:, 1] * img_size / image.shape[0]
        
        labels = [x['category_id'] for x in anns]
        areas = [x['area'] for x in anns]
        
        insert_seg = []
        for x in anns:
            insert_seg.append(x['segmentation'])
        
#         segments = np.array(insert_seg)
        segments = insert_seg
        
        image = cv2.resize(image, (img_size, img_size))
        
        return image, labels, boxes, segments, areas

In [15]:
annotation = '/opt/ml/input/data/train_mosaic_upsampling.json'
data_dir = '/opt/ml/input/data'
img_size = 1024

coco = COCO(annotation)

mosaic_images = []
mosaic_labels = []
mosaic_boxes = []
mosaic_segments = []
mosaic_areas = []

coco = COCO(annotation)

for idx in tqdm(range(3000,3100)):
    image , labels, boxes, segments, areas =  get_img_label(idx, img_size, coco, data_dir )
    mosaic_images.append(image)
    mosaic_labels.append(labels)
    mosaic_boxes.append(boxes)
    mosaic_segments.append(segments)
    mosaic_areas.append(areas)

## load확인 시각화 코드

In [16]:
for i in range(10):
    
    image = mosaic_images[i]
    labels = mosaic_labels[i]
    boxes = mosaic_boxes[i]
    segments = mosaic_segments[i]
    
    plt.figure(figsize=(12,17))
    plt.axis('off')

    plt.imshow(image)

    edge_colors = ['blue', 'brown', 'orange', 'pink', 'green',
                  'gray', 'red', 'olive', 'purple', 'cyan',
                   'black', 'seagreen'
                  ]

    ax = plt.gca()

    for i in range(len(labels)):
        class_label = labels[i]
        boxes_info = boxes[i]
        seg_info = segments[i]

        for seg_info_location in seg_info:
            for i in range(0,len(seg_info_location),2):
                x = seg_info_location[i]
                y = seg_info_location[i+1]

                rect = patches.Rectangle(
                    (x, y), 1, 1,
                    edgecolor='black',
                    facecolor='black',
                    linewidth=0.5,
                    fill=True
                )

                ax.add_patch(rect)


        rect = patches.Rectangle((boxes_info[0],boxes_info[1]),
                             boxes_info[2] - boxes_info[0],
                             boxes_info[3] - boxes_info[1],
                             linewidth=2,
                             edgecolor=edge_colors[class_label],
                             fill = False)

        ax.add_patch(rect)

    plt.show()